# Experiment zhlukovania so všetkými bunkami

**Autor: Bc. Ivan Vykopal**

Pri tomto experimente využívame všetky bunky bez rozdielu klasifikácie.

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob
import numpy as np
from sklearn.cluster import DBSCAN
import geojson
from shapely.geometry import shape
import os

In [ ]:
def display_sample(display_list):
    plt.figure(figsize=(18, 18))

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

In [ ]:
img_directory = 'D:/Master Thesis/ANN Imunne cells/'
geojson_directory = 'D:/Master Thesis/Data/EMB-IKEM-2022-03-09/QuPath project EMB - anotations/annotations/' # upraviť
#out_dir = 'D:/Master Thesis/Code/Clustering/result2/' #upraviť
out_dir = r'C:\Users\ivanv\Desktop\Final tests/'
geojson_suf = '.vsi - 20x.geojson'

In [ ]:
def dilate(mask, original_mask):
    dilated = mask
    nuclei, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    num_dilatations = 0

    while len(nuclei) != 1:
        dilated = cv.dilate(dilated, cv.getStructuringElement(cv.MORPH_ELLIPSE, (10, 10)))
        nuclei, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        num_dilatations += 1

    while num_dilatations:
        dilated = cv.erode(dilated, cv.getStructuringElement(cv.MORPH_ELLIPSE, (10, 10)))
        num_dilatations -= 1

    contours, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(original_mask, contours, -1, (0, 255, 0), 3)
    return original_mask

In [ ]:
def get_nucleus_info(feature):
    attributes = ["Nucleus: Hematoxylin OD mean", "Nucleus: Hematoxylin OD sum", "Nucleus: Hematoxylin OD std dev", "Nucleus: Hematoxylin OD max", "Nucleus: Hematoxylin OD min", "Nucleus: Eosin OD mean", "Nucleus: Eosin OD sum", "Nucleus: Eosin OD std dev", "Nucleus: Eosin OD max", "Nucleus: Eosin OD min", "Nucleus: Eosin OD range"]
    info = list()
    
    try:
        for measurement in feature['properties']['measurements']:
            if measurement['name'] in attributes:
                info.append(measurement['value'])
    except:
        print(feature)
        
    return info

In [ ]:
# Original: eps=100, min_samples=15
# Experiment 2: eps=100, min_samples=20
def get_inflammatory(image_dir, image_name, geojson_path, output_dir, eps=100, min_samples=20):
    image = cv.imread(f"{image_dir}{image_name}.png")
    with open(geojson_path) as f:
        gj = geojson.load(f)
        
    features = gj['features'][1:]
    centroids = list()
    polygons = dict()
    
    index = 0
    for feature in features:
        if feature['properties']['classification']['name'] != 'Region*':
            s = shape(feature['geometry'])
            polygons[index] = s
            #other_features = get_nucleus_info(feature)
            centroids.append([s.centroid.x, s.centroid.y])#, *other_features])
            index += 1
    
    X = np.array(centroids)
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    unique = np.unique(db.labels_)
            
    canvas = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    for unique_idx, unique_value in enumerate(unique[1:]):
        indexes = np.where(db.labels_ == unique_value)[0]
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        for idx in indexes:
            coors = list(zip(*polygons[idx].exterior.coords.xy))
            pts = [[round(c[0]), round(c[1])] for c in coors]
            cv.fillPoly(mask, [np.array(pts)], 1)
            cv.fillPoly(canvas, [np.array(pts)], unique_idx + 1)

        image = dilate(mask, image)
    cv.imwrite(f"{output_dir}{file_name}.png", image)

In [ ]:
files = glob.glob(f"{geojson_directory}*")

for file in files:
    file_name = file.replace('\\', '/').replace(geojson_directory, '').replace(geojson_suf, '')
    print(f'Analyzing {file_name}!')
    if os.path.exists(f"{img_directory}{file_name}.png"):
        image = get_inflammatory(img_directory, file_name, file, out_dir)
    break